In [1]:
import pandas as pd
import numpy as np

IMAGE_WIDTH = 1280
IMAGE_HEIGHT = 960

def normalize(value, max_value):
    return round(float(value) / max_value, 6)

def ellipse_bounding_box(x_center_px, y_center_px, width_px, height_px, angle_deg):
    angle_rad = np.deg2rad(angle_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)

    a = width_px / 2
    b = height_px / 2

    dx = np.sqrt((a * cos_a)**2 + (b * sin_a)**2)
    dy = np.sqrt((a * sin_a)**2 + (b * cos_a)**2)

    x_min = x_center_px - dx
    x_max = x_center_px + dx
    y_min = y_center_px - dy
    y_max = y_center_px + dy

    return x_min, y_min, x_max, y_max

def convert_to_yolo(input_path, output_path, margin=0):
    df = pd.read_csv(input_path)
    yolo_lines = []

    for _, row in df.iterrows():
        try:
            xpix = float(row["xpix."])
            ypix = IMAGE_HEIGHT - float(row["ypix."])
            width_mm = float(row["szerokośćmm"])
            height_mm = float(row["długośćmm"])
            angle = -float(row["kąt"]) + 90

            width_px = width_mm * IMAGE_WIDTH / 128.0
            height_px = height_mm * IMAGE_HEIGHT / 96.0

            x_min, y_min, x_max, y_max = ellipse_bounding_box(xpix, ypix, width_px, height_px, angle)

            # Apply margin
            x_min -= margin
            x_max += margin
            y_min -= margin
            y_max += margin

            # Clamp
            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(IMAGE_WIDTH, x_max)
            y_max = min(IMAGE_HEIGHT, y_max)

            bbox_width = x_max - x_min
            bbox_height = y_max - y_min
            bbox_x_center = x_min + bbox_width / 2
            bbox_y_center = y_min + bbox_height / 2

            yolo_line = f"0 {normalize(bbox_x_center, IMAGE_WIDTH)} {normalize(bbox_y_center, IMAGE_HEIGHT)} {normalize(bbox_width, IMAGE_WIDTH)} {normalize(bbox_height, IMAGE_HEIGHT)}"
            yolo_lines.append(yolo_line)
        except (ValueError, KeyError):
            continue

    with open(output_path, "w", encoding="utf-8") as f:
        f.write("\n".join(yolo_lines))
        print(f"Saved {len(yolo_lines)} YOLO annotations to {output_path}")


In [11]:
# Input and output files
INPUT_FILE = "raport_5_y_2.csv"
OUTPUT_FILE = "dataset0/labels/val/a1_y.txt"
# Run the converter
convert_to_yolo(INPUT_FILE, OUTPUT_FILE,margin=10)

Saved 30 YOLO annotations to dataset0/labels/val/a1_y.txt


In [5]:
import os
import shutil

START_PATH = "data_to_clean/01_Leba_Wiktoria/wrzesień_2023"
letters = ['x','y','z']
numbers = [1,2,3,4,5,6,'6A',7,8,10]


for number in numbers:
    for letter in letters:
        try:
            os.listdir(f"../{START_PATH}/{number}/{letter}")
        except:
            continue
        for second_number in range(1,11):
            dir_with_reports = f"../{START_PATH}/{number}/{letter}"
            dir_with_images = f"../{START_PATH}/{number}/{letter}/a{second_number}"
            

            image_files = [f for f in os.listdir(dir_with_images) if f.endswith('.jpg')]

            if len(image_files) != 1:
                print(f"Skipping {dir_with_images}: found {len(image_files)} images (expected 1)")
                continue
            
            IMAGE_PATH = f"{dir_with_images}/{image_files[0]}"
            report_number = int(IMAGE_PATH.split('/')[-1].split('.')[0].split("_")[1])
            REPORT_PATH = f"{dir_with_reports}/{[f for f in os.listdir(dir_with_reports) if f.endswith(str(report_number) + '.csv')][0]}"
            OUTPUT_FILE_LABEL = f"../dataset0/labels/train/wiktoria_wrzesien_{number}_{letter}_a{second_number}.txt"
            OUTPUT_FILE_IMAGE = f"../dataset0/images/train/wiktoria_wrzesien_{number}_{letter}_a{second_number}.jpg"

            convert_to_yolo(REPORT_PATH, OUTPUT_FILE_LABEL,margin=10)
            shutil.copy(IMAGE_PATH, OUTPUT_FILE_IMAGE)

Saved 9 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a1.txt
Saved 9 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a2.txt
Saved 11 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a3.txt
Saved 18 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a4.txt
Saved 10 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a5.txt
Saved 11 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a6.txt
Saved 0 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a7.txt
Saved 8 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a8.txt
Saved 5 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a9.txt
Saved 5 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_x_a10.txt
Saved 6 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_y_a1.txt
Saved 6 YOLO annotations to ../dataset0/labels/train/wiktoria_wrzesien_1_y_a2.txt
Saved 8 YOL